In [7]:
#Se importan las librerias necesarias para 
import pandas as pd
import numpy as np

EXTRACTING

In [8]:
#Se llaman los archivos y se ingresan a un DataFrame
circuits = pd.read_csv('..\Datasets\circuits.csv')
constructors = pd.read_json('..\Datasets\constructors.json')
drivers = pd.read_json('..\Datasets\drivers.json')
pit_stops = pd.read_json('..\Datasets\pit_stops.json')
races = pd.read_csv('..\Datasets\_races.csv')
results = pd.read_json('..\Datasets\_results.json')

TRANSFORM

In [22]:
#Se modifica la tabla circuits
Mod_circuits=circuits.copy()

#Se elimina los id 852, 853 y 854 por no estar relacionados en la tabla de Races
Mod_circuits.drop([71,73],axis=0,inplace=True)

Mod_circuits.rename(columns={'circuitId':'Circuit_Id','circuitRef':'Circuit_ref',
                    'lat':'Latitud','lng':'Longitud'},inplace=True)
Mod_circuits.drop(['url'],axis=1,inplace=True)
Mod_circuits.columns=Mod_circuits.columns.str.capitalize()
Mod_circuits['Circuit_ref']=Mod_circuits['Circuit_ref'].str.capitalize()

Mod_circuits['Latitud']=Mod_circuits['Latitud'].round(2)
Mod_circuits['Latitud']=Mod_circuits['Longitud'].round(2)



In [12]:
#Se modifica la tabla constructors
Mod_constructors=constructors.copy()



Mod_constructors.rename(columns={'constructorId':'Constructor_Id','constructorRef':'Constructor_ref'},inplace=True)
Mod_constructors.drop(['url'],axis=1,inplace=True)
Mod_constructors.columns=Mod_constructors.columns.str.capitalize()
Mod_constructors['Constructor_ref']=Mod_constructors['Constructor_ref'].str.capitalize()

In [13]:
#Se modifica la tabla drivers

#Se crea la copia con los cambios
Mod_drivers=drivers.copy()

#Se elimina los id 852, 853 y 854 por no estar relacionados en la tabla de Results
Mod_drivers.drop([850,851,852],axis=0,inplace=True)

#Se elimina columna Number por tener mayoría de vacíos y no estar relacionada
Mod_drivers.drop(['number'],axis=1,inplace=True)

#Se completan los valores de code con las 3 primeras letras del driverRef
Mod_drivers['code']=Mod_drivers['driverRef'].str[:3].str.upper()

#Se extrae el nombre del formato diccionario
drivers.name
Forename=[]
Surname=[]
for i in Mod_drivers.name:
    valores=list(i.values())
    Forename.append(valores[0])
    Surname.append(valores[1])

Mod_drivers['Forename']=Forename
Mod_drivers['Surname']=Surname
Mod_drivers.drop(['name'],axis=1,inplace=True)

#Se modifica el tipo dob a date
Mod_drivers['dob']=pd.to_datetime(Mod_drivers['dob'])

#Se elimina la columna url
Mod_drivers.drop(['url'],axis=1,inplace=True)

#Se organiza y estandariza los datos de las columnas
Mod_drivers=Mod_drivers.loc[:,["driverId","driverRef","code","Forename","Surname","nationality","dob"]]

Mod_drivers.rename(columns={'driverId':'Driver_Id','driverRef':'Driver_ref'},inplace=True)
Mod_drivers.columns=Mod_drivers.columns.str.capitalize()

Mod_drivers['Driver_ref']=Mod_drivers['Driver_ref'].str.capitalize()

In [14]:
#Se modifica la tabla pit_stops

#Se crea la copia con los cambios
from operator import index


Mod_pit_stops=pit_stops.copy()

#Se modifica el tipo dob a date
#Mod_pit_stops['time']=pd.to_timedelta(Mod_pit_stops['time'])

#Se crea una nueva columna
Mod_pit_stops['Pits_id']=Mod_pit_stops['raceId']*10000+Mod_pit_stops['driverId']*10+Mod_pit_stops['stop']

#Se re-organizan las columnas
cols = Mod_pit_stops.columns.tolist()
cols = cols[-1:] + cols[:-1]
Mod_pit_stops = Mod_pit_stops[cols]

#Se estandariza los nombres de las columnas
Mod_pit_stops.rename(columns={'raceId':'Race_Id','driverId':'Driver_id',
                            'duration':'duration_s','milliseconds':'duration_ms'},inplace=True)
Mod_pit_stops.columns=Mod_pit_stops.columns.str.capitalize()

#Se arregla formato a segundos
for i in Mod_pit_stops.index:
    #print(i)
    if type(Mod_pit_stops.loc[i,'Duration_s']) != int:
        Mod_pit_stops.loc[i,'Duration_s']=Mod_pit_stops.loc[i,'Duration_ms']/1000


In [23]:
#Se modifica la tabla races

#Se crea la copia con los cambios
Mod_races=races.copy()

#Se modifica el tipo dob a date
Mod_races['date']=Mod_races['date'].str.replace("/","-")
Mod_races['date']=pd.to_datetime(Mod_races['date'])

#Se elimina la columna url y time
Mod_races.drop(['url'],axis=1,inplace=True)
Mod_races.drop(['time'],axis=1,inplace=True)

#Se organiza y estandariza los datos de las columnas
Mod_races.rename(columns={'raceId':'race_Id','circuitId':'circuit_id'},inplace=True)
Mod_races.columns=Mod_races.columns.str.capitalize()



C:\Users\Juanita\AppData\Local\Temp\ipykernel_3408\3942526714.py:8: UserWarning: Parsing '29-03-2009' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Mod_races['date']=pd.to_datetime(Mod_races['date'])
C:\Users\Juanita\AppData\Local\Temp\ipykernel_3408\3942526714.py:8: UserWarning: Parsing '19-04-2009' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Mod_races['date']=pd.to_datetime(Mod_races['date'])
C:\Users\Juanita\AppData\Local\Temp\ipykernel_3408\3942526714.py:8: UserWarning: Parsing '26-04-2009' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Mod_races['date']=pd.to_datetime(Mod_races['date'])
C:\Users\Juanita\AppData\Local\Temp\ipykernel_3408\3942526714.py:8: UserWarning: Parsing '24-05-2009' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Mod_races['date']=pd.to_datetime

In [16]:
#Se modifica la tabla results

#Se crea la copia con los cambios
Mod_results=results.copy()

#Se identifican los valores faltantes y se les asigna un None

Mod_results['fastestLapTime'].replace(to_replace="Vacio",value=None,inplace=True)
Mod_results['fastestLapSpeed'].replace(to_replace="Vacio",value=None,inplace=True)
Mod_results['fastestLap'].replace(to_replace="Vacio",value=None,inplace=True)
Mod_results['rank'].replace(to_replace="Vacio",value=None,inplace=True)
Mod_results['number'].replace(to_replace="Vacio",value=None,inplace=True)


#Se elimina la columna position, position text por esta asociado a la columna positionOrder
#Se elimina times y milisec por falta de datos
Mod_results.drop(['position'],axis=1,inplace=True)
Mod_results.drop(['positionText'],axis=1,inplace=True)
Mod_results.drop(['time'],axis=1,inplace=True)
Mod_results.drop(['milliseconds'],axis=1,inplace=True)

#Se organiza y estandariza los datos de las columnas
Mod_results.rename(columns={'resultId':'result_Id','raceId':'race_Id','driverId':'driver_Id'},inplace=True)
Mod_results.columns=Mod_results.columns.str.capitalize()

In [21]:
A=list(Mod_races['Circuit_ref'])
B=list(Mod_circuits['Circuit_id'])

for i in B:
    if i in A:
        continue
    else:
        print(i)


72
74


LOAD

In [24]:
import pyodbc
from operator import contains

In [25]:
sql_conn=pyodbc.connect(
                        'DRIVER={MySQL ODBC 8.0 ANSI Driver};'
                        'SERVER=localhost;'
                        'UID=root;'
                        'PASSWORD=vW4d4ijJblS2!;'
                        'Database=pi_database;'
)

In [26]:
cursor=sql_conn.cursor()


In [29]:
sql_execute="DROP TABLE races"
cursor.execute(sql_execute)

In [8]:
 # Creamos las tablas en SQL

 #Circuits

sql_create=("CREATE TABLE Circuits" 
            "(Circuit_id int," 
            "Circuit_ref varchar(32)," 
            "Name varchar(80),"
            "Location varchar(32),"
            "Country varchar(32),"
            "Latitud float,"
            "Longitud float,"
            "Alt int)")
cursor.execute(sql_create)

#Constructors
sql_create=("CREATE TABLE Constructors" 
            "(Constructor_id int," 
            "Constructor_ref varchar(32)," 
            "Name varchar(32),"
            "Nationality varchar(32))")
cursor.execute(sql_create)

#Drivers
sql_create=("CREATE TABLE Drivers" 
            "(Driver_id int," 
            "Driver_ref varchar(32)," 
            "Code varchar(32),"
            "Forename varchar(32),"
            "Surname varchar(32),"
            "Nationality varchar(32),"
            "Dob Date)")
cursor.execute(sql_create)

#Pit_stops
sql_create=("CREATE TABLE Pit_stops" 
            "(Pits_id int," 
            "Race_id int," 
            "Driver_id int,"
            "Stop int,"
            "Lap int,"
            "Time Time,"
            "Duration_s float,"
            "Duration_ms float)")
cursor.execute(sql_create)

#races
sql_create=("CREATE TABLE Races" 
            "(Race_id int," 
            "Year int," 
            "Round int,"
            "Circuit_id int,"
            "Name varchar(50),"
            "Date Date)")
cursor.execute(sql_create)

#Results

sql_create=("CREATE TABLE Results" 
            "(Result_id int," 
            "Race_id int," 
            "Driver_id int,"
            "Constructor_id int,"
            "Number int,"
            "Grid int,"
            "Positionorder int,"
            "Points int,"
            "Laps int,"
            "F_lap int,"
            "F_Rank int,"
            "F_Time Time,"
            "F_Speed float,"
            "StatusId int)")
cursor.execute(sql_create)

In [ ]:
#Cargamos datos de las tablas 

#Circuits
insert_query=("INSERT INTO Circuits"
             "(Circuit_id, Circuit_id, Name, Location,Country,Latitud,Longitud,Alt)" 
             "VALUES (?,?,?,?,?,?,?,?);")
             
Arreglo=np.asarray(Mod_circuits)

for i in Arreglo:
    valores=(i[0],i[1],i[2],i[3],i[4],i[5],i[6],i[7])
                 
    cursor.execute(insert_query,valores)
    
#Constructors
insert_query=("INSERT INTO Constructors"
             "(Constructor_id, Constructor_ref, Name, Nationality)" 
             "VALUES (?,?,?,?);")


Arreglo=np.asarray(Mod_constructors)

for i in Arreglo:
    valores=(i[0],i[1],i[2],i[3])
                 
    cursor.execute(insert_query,valores)

#Drivers

insert_query=("INSERT INTO Drivers"
             "(Driver_id, Driver_ref, Code, Forename,Surname,Nationality,Dob)" 
             "VALUES (?,?,?,?,?,?,?);")
             
Arreglo=np.asarray(Mod_drivers)

for i in Arreglo:
    valores=(i[0],i[1],i[2],i[3],i[4],i[5],i[6])
                 
    cursor.execute(insert_query,valores)

#Pit_Stops

insert_query=("INSERT INTO Pit_stops"
             "(Pits_id, Race_id, Driver_id, Stop,Lap,Time,Duration_s,Duration_ms)" 
             "VALUES (?,?,?,?,?,?,?,?);")
             
Arreglo=np.asarray(Mod_pit_stops)

contador=0
for i in Arreglo:

    valores=(i[0],i[1],i[2],i[3],i[4],i[5],i[6],i[7])
    contador+=1
                 
    cursor.execute(insert_query,valores)

#races

insert_query=("INSERT INTO Races"
             "(Race_id, Year, Round, Circuit_id, Name , Date)" 
             "VALUES (?,?,?,?,?,?);")
             
Arreglo=np.asarray(Mod_races)

contador=0
for i in Arreglo:

    valores=(i[0],i[1],i[2],i[3],i[4],i[5])
    contador+=1
                 
    cursor.execute(insert_query,valores)

#results 

insert_query=("INSERT INTO Results"
             "(Result_id, Race_id, Driver_id, Constructor_id, Number , Grid,"
             "Positionorder,Points,Laps,F_lap,F_Rank,F_Time,F_Speed,StatusId)"
             "VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?);")
             
Arreglo=np.asarray(Mod_results)

contador=0
for i in Arreglo:
    valores=(i[0],i[1],i[2],i[3],i[4],i[5],i[6],i[7],i[8],i[9],i[10],i[11],i[12],i[13])
    contador+=1
                 
    cursor.execute(insert_query,valores)

In [37]:
#Commit para subir los datos
sql_conn.commit()

In [38]:
#Creamos las PK
pk_circuits=("ALTER TABLE Circuits ADD PRIMARY KEY (Circuit_id)")
cursor.execute(pk_circuits)

pk_constructors=("ALTER TABLE Constructors ADD PRIMARY KEY (Constructor_id)")
cursor.execute(pk_constructors)

pk_drivers=("ALTER TABLE Drivers ADD PRIMARY KEY (Driver_id)")
cursor.execute(pk_drivers)

pk_pits=("ALTER TABLE pit_stops ADD PRIMARY KEY (Pits_id)")
cursor.execute(pk_pits)

pk_races=("ALTER TABLE races ADD PRIMARY KEY (race_id)")
cursor.execute(pk_races)

pk_results=("ALTER TABLE results ADD PRIMARY KEY (result_id)")
cursor.execute(pk_results)

In [39]:
#Creamos las relaciones FK
fk_input=("ALTER TABLE Results ADD CONSTRAINT FK_Race FOREIGN KEY (Race_id) REFERENCES races(Race_id)")
cursor.execute(fk_input)

fk_input=("ALTER TABLE Results ADD CONSTRAINT FK_Driver FOREIGN KEY (Driver_id) REFERENCES drivers(Driver_id)")
cursor.execute(fk_input)

fk_input=("ALTER TABLE Results ADD CONSTRAINT FK_Constructors FOREIGN KEY (Constructor_id) REFERENCES constructors(Constructor_id)")
cursor.execute(fk_input)

fk_input=("ALTER TABLE Pit_stops ADD CONSTRAINT FK_pit_race FOREIGN KEY (Race_id) REFERENCES races(Race_id)")
cursor.execute(fk_input)

fk_input=("ALTER TABLE Pit_stops ADD CONSTRAINT FK_pit_driver FOREIGN KEY (Driver_id) REFERENCES drivers(Driver_id)")
cursor.execute(fk_input)

fk_input=("ALTER TABLE Races ADD CONSTRAINT FK_circuit FOREIGN KEY (Circuit_id) REFERENCES circuits(Circuit_id)")
cursor.execute(fk_input)

In [16]:
cursor.close()
sql_conn.close()
